In [42]:
#Instalando o Seleniumbase e pandas
!pip install seleniumbase

#Instalando o Chromium no Colab pois é necessário para a extração do robô
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install -y google-chrome-stable

#Importando as bibliotecas
import json
from seleniumbase import SB
from selenium.webdriver.common.by import By

OK
Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt

In [53]:
# |Constantes e função de login|
URL_SITE = "https://www.saucedemo.com/"
USERNAME = "standard_user"
PASSWORD = "secret_sauce"

CHECKOUT = {
    "first_name": "Trainee",
    "last_name": "PiJunior",
    "zip_code": "31270-901"
}

def realizarLogin(sb: SB):
   #A função automatiza o login no site e recebe o 'sb'para controle do navegador
    print("Iniciando...")
    #Abrir a página usando URL_SITE
    sb.open(URL_SITE)

    # |Preenchimento de credenciais|
    sb.type("#user-name", USERNAME)
    sb.type("#password", PASSWORD)
    sb.click("#login-button")

    # |Validação do login|
    sb.wait_for_element("#inventory_container")
    print("Login realizado com sucesso!")

In [54]:
def extrairDadosProdutos(sb: SB) -> list[dict]:
    # Lista para armazenar todos os produtos
    lista_produtos = []

    # Encontrar todos os contêineres de produtos
    elementos_produtos = sb.find_elements("div.inventory_item")

    print(f"Encontrados {len(elementos_produtos)} produtos.")

    # Iterar sobre os elementos e extrair os dados
    for produto_elemento in elementos_produtos:
        try:
            # Extraindo dados
            nome = produto_elemento.find_element(By.CLASS_NAME, "inventory_item_name").text
            descricao = produto_elemento.find_element(By.CLASS_NAME, "inventory_item_desc").text
            preco_texto = produto_elemento.find_element(By.CLASS_NAME, "inventory_item_price").text

            # Limpar o símbolo da moeda e converter para float
            preco_float = float(preco_texto.replace("$", ""))

            # Adicionar ao dicionário
            produto_dict = {
                "nome": nome,
                "descricao": descricao,
                "preco": preco_float
            }
            lista_produtos.append(produto_dict)

        except Exception as e:
            print(f"Erro ao extrair dados de um produto: {e}")
            continue

    print("Extração de dados concluída!")
    return lista_produtos


In [55]:
# Teste de compilação
def testar_e_imprimir_extracao():

    dados_extraidos = []

    # Inicia o SeleniumBase.
    try:
        with SB(headless=False) as sb:

            # Login
            realizarLogin(sb)

            # Extração de Dados
            dados_extraidos = extrairDadosProdutos(sb)

            # IMPRESSÃO DOS RESULTADOS
            print("Resultado da extração")
            print(f"Total de produtos coletados: {len(dados_extraidos)}")

            if dados_extraidos:
                # O indent=4 torna a estrutura visualmente organizada
                print(json.dumps(dados_extraidos, indent=4, ensure_ascii=False))
                # Verificação rápida de sucesso
                print("\nTeste de extração concluído!")
            else:
                print("Falha: lista de produtos está vazia. Verifique a função extrairDadosProdutos.")

    except Exception as e:
        print(f"\nErro fatal durante o teste {e}")

# chamada principal
testar_e_imprimir_extracao()

Iniciando...
Login realizado com sucesso!
Encontrados 6 produtos.
Extração de dados concluída!
Resultado da extração
Total de produtos coletados: 6
[
    {
        "nome": "Sauce Labs Backpack",
        "descricao": "carry.allTheThings() with the sleek, streamlined Sly Pack that melds uncompromising style with unequaled laptop and tablet protection.",
        "preco": 29.99
    },
    {
        "nome": "Sauce Labs Bike Light",
        "descricao": "A red light isn't the desired state in testing but it sure helps when riding your bike at night. Water-resistant with 3 lighting modes, 1 AAA battery included.",
        "preco": 9.99
    },
    {
        "nome": "Sauce Labs Bolt T-Shirt",
        "descricao": "Get your testing superhero on with the Sauce Labs bolt T-shirt. From American Apparel, 100% ringspun combed cotton, heather gray with red bolt.",
        "preco": 15.99
    },
    {
        "nome": "Sauce Labs Fleece Jacket",
        "descricao": "It's not every day that you come acro

In [57]:
try:
  with SB(headless=True) as sb:
    realizarLogin(sb)

    # Adiciona todos os produtos ao carrinho
    botoes = sb.find_elements(By.CLASS_NAME, "btn_primary")
    for botao in botoes:
      botao.click()
    print("\nTodos os produtos adicionados ao carrinho.")

    # Abre a página do carrinho
    sb.find_element(By.CLASS_NAME, "shopping_cart_link").click()
    sb.wait_for_element("#cart_contents_container")

    # Conclui o check dos itens
    sb.find_element(By.ID, "checkout").click()
    sb.wait_for_element("#checkout_info_container")

    # Adiciona informações referente ao comprador
    sb.find_element(By.ID, "first-name").send_keys(CHECKOUT["first_name"])
    sb.find_element(By.ID, "last-name").send_keys(CHECKOUT["last_name"])
    sb.find_element(By.ID, "postal-code").send_keys(CHECKOUT["zip_code"])

    # Confirma informações
    sb.find_element(By.ID, "continue").click()
    sb.wait_for_element("#checkout_summary_container")

    # Pega os dados referentes ao pagamento e entrega
    dados_pagamento = sb.find_elements(By.CLASS_NAME, "summary_value_label")
    print("Meio de pagamento:", dados_pagamento[0].text)
    print("Forma de entrega:", dados_pagamento[1].text)

    # Raspa o valor total da compra
    valor_total_compra = sb.find_element(By.CLASS_NAME, "summary_total_label").text
    print(f"Valor Total Final: {valor_total_compra}")

    #Finaliza a compra
    sb.find_element(By.ID, "finish").click()

    #Verifica se a compra foi concluída com sucesso
    sb.wait_for_element("#checkout_complete_container")

    #Raspar a mensagem de confirmação
    mensagem_sucesso = sb.find_element(By.CLASS_NAME, "complete-text").text

    if mensagem_sucesso:
      print("Compra finalizada!")
      print("Mensagem de Confirmação:", mensagem_sucesso)
    else:
      print("Erro ao finalizar a compra")

except Exception as e:
 print(f"\nErro ao tentar realizar a compra no bloco principal: {e}")

Iniciando...
Login realizado com sucesso!

Todos os produtos adicionados ao carrinho.
Meio de pagamento: SauceCard #31337
Forma de entrega: Free Pony Express Delivery!
Valor Total Final: Total: $140.34
Compra finalizada!
Mensagem de Confirmação: Your order has been dispatched, and will arrive just as fast as the pony can get there!


In [61]:
#Instalando o Seleniumbase e pandas
!pip install seleniumbase

#Instalando o Chromium no Colab pois é necessário para a extração do robô
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install -y google-chrome-stable

#Importando as bibliotecas
import json
from seleniumbase import SB
from selenium.webdriver.common.by import By

# |Constantes e função de login|
URL_SITE = "https://www.saucedemo.com/"
USERNAME = "standard_user"
PASSWORD = "secret_sauce"

CHECKOUT = {
    "first_name": "Trainee",
    "last_name": "PiJunior",
    "zip_code": "31270-901"
}

def realizarLogin(sb: SB):
    #A função automatiza o login no site e recebe o 'sb'para controle do navegador
    print("Iniciando...")
    #Abrir a página usando URL_SITE
    sb.open(URL_SITE)

    # |Preenchimento de credenciais|
    sb.type("#user-name", USERNAME)
    sb.type("#password", PASSWORD)
    sb.click("#login-button")

    # |Validação do login|
    sb.wait_for_element("#inventory_container")
    print("Login realizado com sucesso!")

def extrairDadosProdutos(sb: SB) -> list[dict]:
    # Lista para armazenar todos os produtos
    lista_produtos = []

    # Encontrar todos os contêineres de produtos
    elementos_produtos = sb.find_elements("div.inventory_item")

    print(f"Encontrados {len(elementos_produtos)} produtos.")

    # Iterar sobre os elementos e extrair os dados
    for produto_elemento in elementos_produtos:
        try:
            # Extraindo dados
            nome = produto_elemento.find_element(By.CLASS_NAME, "inventory_item_name").text
            descricao = produto_elemento.find_element(By.CLASS_NAME, "inventory_item_desc").text
            preco_texto = produto_elemento.find_element(By.CLASS_NAME, "inventory_item_price").text

            # Limpar o símbolo da moeda e converter para float
            preco_float = float(preco_texto.replace("$", ""))

            # Adicionar ao dicionário
            produto_dict = {
                "nome": nome,
                "descricao": descricao,
                "preco": preco_float
            }
            lista_produtos.append(produto_dict)

        except Exception as e:
            print(f"Erro ao extrair dados de um produto: {e}")
            continue

    print("Extração de dados concluída!")
    return lista_produtos

# -------------------------------------------------------------
# NOVO: Função para gerar e salvar o arquivo JSON
# -------------------------------------------------------------
def gerar_json_final(dados: list[dict], nome_arquivo: str):
    """Salva a lista de dicionários em um arquivo JSON."""
    try:
        with open(nome_arquivo, 'w', encoding='utf-8') as f:
            json.dump(dados, f, indent=4, ensure_ascii=False)
        print(f"\n✅ Dados da compra salvos com sucesso em **{nome_arquivo}**")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo JSON: {e}")
# -------------------------------------------------------------

# Teste de compilação
def testar_e_imprimir_extracao():

    dados_extraidos = []

    # Inicia o SeleniumBase.
    try:
        with SB(headless=False) as sb:

            # Login
            realizarLogin(sb)

            # Extração de Dados
            dados_extraidos = extrairDadosProdutos(sb)

            # IMPRESSÃO DOS RESULTADOS
            print("Resultado da extração")
            print(f"Total de produtos coletados: {len(dados_extraidos)}")

            if dados_extraidos:
                # O indent=4 torna a estrutura visualmente organizada
                print(json.dumps(dados_extraidos, indent=4, ensure_ascii=False))
                # Verificação rápida de sucesso
                print("\nTeste de extração concluído!")
            else:
                print("Falha: lista de produtos está vazia. Verifique a função extrairDadosProdutos.")

    except Exception as e:
        print(f"\nErro fatal durante o teste {e}")

# chamada principal
testar_e_imprimir_extracao()

# -------------------------------------------------------------
# NOVO: Variável para armazenar os dados finais do checkout
# -------------------------------------------------------------
dados_finais_compra = []

try:
    with SB(headless=True) as sb:
        realizarLogin(sb)

        # Adiciona todos os produtos ao carrinho
        botoes = sb.find_elements(By.CLASS_NAME, "btn_primary")
        for botao in botoes:
            botao.click()
        print("\nTodos os produtos adicionados ao carrinho.")

        # Abre a página do carrinho
        sb.find_element(By.CLASS_NAME, "shopping_cart_link").click()
        sb.wait_for_element("#cart_contents_container")

        # Conclui o check dos itens
        sb.find_element(By.ID, "checkout").click()
        sb.wait_for_element("#checkout_info_container")

        # Adiciona informações referente ao comprador
        sb.find_element(By.ID, "first-name").send_keys(CHECKOUT["first_name"])
        sb.find_element(By.ID, "last-name").send_keys(CHECKOUT["last_name"])
        sb.find_element(By.ID, "postal-code").send_keys(CHECKOUT["zip_code"])

        # Confirma informações
        sb.find_element(By.ID, "continue").click()
        sb.wait_for_element("#checkout_summary_container")

        # Pega os dados referentes ao pagamento e entrega
        dados_pagamento = sb.find_elements(By.CLASS_NAME, "summary_value_label")
        # NOVO: Salva em variáveis para uso no JSON
        meio_pagamento = dados_pagamento[0].text
        forma_entrega = dados_pagamento[1].text
        print("Meio de pagamento:", meio_pagamento)
        print("Forma de entrega:", forma_entrega)

        # Raspa o valor total da compra
        # NOVO: Salva em variável para uso no JSON
        valor_total_compra_texto = sb.find_element(By.CLASS_NAME, "summary_total_label").text
        print(f"Valor Total Final: {valor_total_compra_texto}")

        #Finaliza a compra
        sb.find_element(By.ID, "finish").click()

        #Verifica se a compra foi concluída com sucesso
        sb.wait_for_element("#checkout_complete_container")

        #Raspar a mensagem de confirmação
        mensagem_sucesso = sb.find_element(By.CLASS_NAME, "complete-text").text

        if mensagem_sucesso:
            print("Compra finalizada!")
            print("Mensagem de Confirmação:", mensagem_sucesso)

            # NOVO: Adiciona os dados do checkout à lista
            dados_finais_compra.append({
                "status_compra": "Concluída com Sucesso",
                "meio_pagamento": meio_pagamento,
                "forma_entrega": forma_entrega,
                "valor_total": valor_total_compra_texto,
                "mensagem_confirmacao": mensagem_sucesso
            })
        else:
            print("Erro ao finalizar a compra")

        # NOVO: CHAMADA PARA GERAR O ARQUIVO JSON
        if dados_finais_compra:
            gerar_json_final(dados_finais_compra, "resumo_compra_final.json")

except Exception as e:
    print(f"\nErro ao tentar realizar a compra no bloco principal: {e}")

OK
Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://cli.github.com/packages stable InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt